In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import re

In [ ]:
vn_names = pd.read_excel()

In [2]:
s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
def remove_accents(input_str):
	s = ''
	# print(input_str.encode('utf-8'))
	for c in input_str:
		if c in s1:
			s += s0[s1.index(c)]
		else:
			s += c
	return s

In [31]:
# patron = pd.read_csv('patron_profile.tab', encoding= 'latin-1', sep= '\t')
patron = pd.read_csv('patron_profile_V2.tab', encoding= 'latin-1', sep= '\t')
patron.columns = [x for x in patron.columns.str.lower()]
lst = ['memberid', 'gender', 'firstname', 'lastname', 'country', 'nationality']
df = patron[lst]

In [32]:
df['blank_nationality'] = (df['nationality'].isna()).astype(int)

lst_last_name = pd.read_excel('vietnamese_family_name.xlsx', sheet_name= 'Sheet1')[["LNAME", "PCT"]]
# df['is_lastname'] = df['lastname'].apply(lambda x: 1 if x in lst_last_name["LNAME"].values else 0)
pct_lname = lst_last_name.groupby("LNAME")["PCT"].sum().reset_index()
pct_lname.rename(columns= {'LNAME':'lastname', "PCT": 'pct'}, inplace= True)
df = df.merge(pct_lname, on= 'lastname', how= 'left')
df["pct"].fillna(0, inplace= True)
# df['is_male'] = (df['gender'] == 'M').astype(int)

C:\Users\iris.ha\AppData\Local\Temp\ipykernel_9836\321261443.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['blank_nationality'] = (df['nationality'].isna()).astype(int)


In [34]:
from transform import vnword1, vnword2, vnword3, vnword4, vnword5, notvnword, contains_ln

In [35]:
# df['vn_firstname'] = df['firstname'].apply(lambda x: vnword1(str(x).lower()))
df['is_word1'] = (df['firstname'].apply(lambda x: len(vnword1(str(x).lower()))) >=1).astype(int)
df['is_word2'] = (df['firstname'].apply(lambda x: len(vnword2(str(x).lower()))) >=1).astype(int)
df['is_word3'] = (df['firstname'].apply(lambda x: len(vnword3(str(x).lower()))) >=1).astype(int)
df['is_word4'] = (df['firstname'].apply(lambda x: len(vnword4(str(x).lower()))) >=1).astype(int)
df['not_word'] = (df['firstname'].apply(lambda x: len(notvnword(str(x).lower()))) >=1).astype(int)
df['is_word1_ln'] = (df['lastname'].apply(lambda x: len(vnword1(str(x).lower()))) >=1).astype(int)
df['is_word2_ln'] = (df['lastname'].apply(lambda x: len(vnword2(str(x).lower()))) >=1).astype(int)
df['is_word3_ln'] = (df['lastname'].apply(lambda x: len(vnword3(str(x).lower()))) >=1).astype(int)
df['is_word4_ln'] = (df['lastname'].apply(lambda x: len(vnword4(str(x).lower()))) >=1).astype(int)
df['not_word_ln'] = (df['lastname'].apply(lambda x: len(notvnword(str(x).lower()))) >=1).astype(int)

df['contain_ln_ln'] = df['lastname'].apply(lambda x: contains_ln(str(x), lst_last_name["LNAME"].values))
df['contain_ln_fn'] = df['firstname'].apply(lambda x: contains_ln(str(x), lst_last_name["LNAME"].values))

df['is_korean'] = df['country'].apply(lambda x: 1 if 'korea' in str(x).lower() else 0)
df['is_chinese'] = df['nationality'].apply(lambda x: 1 if 'chinese' in str(x).lower() else 0)
df['is_hongkongese'] = df['nationality'].apply(lambda x: 1 if 'hong kong' in str(x).lower() else 0)
df['is_singaporen'] = df['country'].apply(lambda x: 1 if 'singapore' in str(x).lower() else 0)
df['is_malaysian'] = df['country'].apply(lambda x: 1 if 'malaysia' in str(x).lower() else 0)

In [36]:
df

,memberid,gender,firstname,lastname,country,nationality,blank_nationality,pct,is_word1,is_word2,...,is_word3_ln,is_word4_ln,not_word_ln,contain_ln_ln,contain_ln_fn,is_korean,is_chinese,is_hongkongese,is_singaporen,is_malaysian
0,10000001,M,LIM,SON,Cambodia,Cambodian,0,0.00042,1,0,...,0,0,0,1,0,0,0,0,0,0
1,10000002,F,YU HSUEH,YANG,Taiwan,"Chinese, Taiwanese",0,0.00000,0,0,...,0,0,1,0,0,0,1,0,0,0
2,10000003,M,MYUNG WON,KIM,Korea (Republic of),South Korean,0,0.00064,0,0,...,0,0,0,1,0,1,0,0,0,0
3,10000004,M,DAE JUNG,KWON,Korea (Republic of),South Korean,0,0.00000,0,0,...,0,0,1,0,0,1,0,0,0,0
4,10000005,M,JEONG HO,BAEK,Korea (Republic of),South Korean,0,0.00000,1,0,...,0,0,1,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124477,80015043,M,ELFIN,ELFIN,Indonesia,Indonesian,0,0.00000,0,0,...,0,0,1,0,0,0,0,0,0,0
124478,80015044,M,WE,MIN,Indonesia,Indonesian,0,0.00000,0,0,...,0,0,0,0,0,0,0,0,0,0
124479,80015045,M,CHIANG,SING FOO,Singapore,Singaporean,0,0.00000,0,0,...,0,0,1,0,0,0,0,0,1,0
124480,80015046,M,LONGSHOU,YIN,China,Chinese,0,0.00000,0,0,...,0,0,1,0,0,0,1,0,0,0


In [37]:
# df[(df['is_firstname'] >= 1)*(df['is_lastname'] == 1)].to_csv('result.csv', index= False)
df.to_csv('result_check.txt', sep= ';', index= False)